In [1]:
# !unzip gymhearts.zip
# %cd robohearts/

In [2]:
import gym
import numpy
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.agent_random import RandomAgent
from gymhearts.Agent.agent_reinforce import REINFORCE_Agent
from gymhearts.Agent.utils_env import *
from gymhearts.Agent.utils_nn import *
from tqdm import tqdm_notebook
from copy import deepcopy

In [3]:
# ----------- TRAINING CONFIGURATION -------------
# Number of games to train on 
TRAINING_ITERS = 5

# Number of episodes to run during model evaluation
NUM_EPISODES = 100

# Number of model evaluations to average together
NUM_TESTS = 10

# Max score for players to win the game
MAX_SCORE = 100

# Batch size for performing updates
BATCH_SIZE = 16

# Set to false to skip training
run_train = False

# Set to true to resume training from an existing model
continue_train = False

# Run testing on a random agent for comparison
run_random = False

# Features to include in model :: [in_hand, in_play, played_cards, won_cards, scores]
feature_list = ['in_hand', 'in_play']

# Name of the file that is saved :: {model_name}_pi.th, {model_name}_v.th
model_name = 'reinforce'

# Configuration parameters for the model
reinforce_config = {
    'print_info' : False,
    'epsilon' : .01,
    'gamma' : 1,
    'alpha': 3e-6,
    'feature_list' : feature_list,
}

if continue_train:
    mc_nn_config['load_model'] = model_name

playersNameList = ['REINFORCE', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

agent_list[0] = REINFORCE_Agent(playersNameList[0], reinforce_config)
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [4]:
# ---------- TRAIN REINFORCE AGENT ------------
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
if run_train:
    returns = []
    for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
         # Save the model every 50 steps
        if trn_episode % 10 == 0:
            save_model(agent_list[0].baseline.nn, model_name, 'reinforce', agent_list[0].pi.nn)
        batch = []
        for i in range(BATCH_SIZE):
            observation = env.reset()
            episode_history = []
            while True:
                #env.render()
                now_event = observation['event_name']
                IsBroadcast = observation['broadcast']
                
                if now_event == 'RoundEnd' and len(episode_history) > 0:
                    # Add episode info and return to batch
                    batch.append((deepcopy(episode_history), -reward['REINFORCE']))
                    episode_history = []
                    
                if now_event == 'GameOver':
                    break

                action = None
                if IsBroadcast == True:
                    for agent in agent_list:
                        agent.Do_Action(observation)
                else:
                    playName = observation['data']['playerName']
                    for agent in agent_list:
                        if agent.name == playName:
                            action = agent.Do_Action(observation)

                observation_prime, reward, done, info = env.step(action)
                
                if observation['event_name'] == 'PlayTrick' and observation['data']['playerName'] == 'REINFORCE':
                    state_features, valid_features = agent_list[0].generate_features(observation)
                    episode_history.append((state_features, valid_features, action))
                    
                observation = observation_prime
        returns.append(agent_list[0].update(batch))
        
    save_model(agent_list[0].baseline.nn, model_name, 'reinforce', agent_list[0].pi.nn)
    #avg_returns = numpy.array(returns)
    #numpy.savetxt('reinforce.txt', avg_returns)\n"

In [5]:
# Function to test reinforce model with multiprocessing
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'REINFORCE')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [6]:
# ----------- EVALUATE REINFORCE AGENT ---------------

env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)

# Evaluation parameters for testing
reinforce_config = {
    'print_info' : False,
    'load_model' : model_name,
    'feature_list' : feature_list
}

agent_list[0] = REINFORCE_Agent(playersNameList[0], reinforce_config)
reinforce_wins = [0] * NUM_TESTS

pool = multiprocessing.Pool(processes=NUM_TESTS)
reinforce_wins = pool.map(run_test, reinforce_wins)
print(reinforce_wins)
pool.close()
pool.join()











[18, 26, 30, 21, 21, 27, 33, 18, 24, 19]


In [7]:
# Function to test random model with multiprocessing
def run_test_random(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [8]:
# ----------- EVALUTATE RANDOM AGENT ---------------
if run_random:
    env = gym.make('Hearts_Card_Game-v0')
    env.__init__(playersNameList, MAX_SCORE)
    playersNameList[0] = 'Randman'
    agent_list[0] = RandomAgent(playersNameList[0])
    rand_wins = [0] * NUM_TESTS
    pool = multiprocessing.Pool(processes=NUM_TESTS)
    rand_wins = pool.map(run_test_random, rand_wins)
    print(rand_wins)
    pool.close()
    pool.join()

In [9]:
print(f"REINFORCE won {sum(reinforce_wins)/len(reinforce_wins)} times on average :: {str(reinforce_wins)}")
for wins in reinforce_wins:
    print(wins)
if run_random:
    print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")

REINFORCE won 23.7 times on average :: [18, 26, 30, 21, 21, 27, 33, 18, 24, 19]
18
26
30
21
21
27
33
18
24
19
